#### Подготовка данных

Изначальный датасет довольно избыточный, поэтому для проведения задания мы пользуемся этим ноутбуком, чтобы взять только нужную часть

Скачать весь датасет (~5 Гб) можно по этой ссылке: https://drive.google.com/file/d/1CUZnBtYwifVXS21yVg62T-vrPVayso5H/view
Файл с отсутствующими скелетами: https://github.com/shahroudy/NTURGB-D/blob/master/Matlab/NTU_RGBD_samples_with_missing_skeletons.txt (т.е. список файлов из большого датасета, в которых отсутсвуют скелеты и мы эти файлы будем игнорировать)

In [15]:
import numpy as np

In [16]:
data_path = "nturgbd_skeletons_s001_to_s017/"
data_path = "nturgbd_skeletons_s001_to_s017/nturgb+d_skeletons/"

#### список отсутсвующих элементов так же будет доступен 
broken_files_path = "NTU_RGBD_samples_with_missing_skeletons.txt"

In [17]:
training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = [3, 7, 24, 38, 27, 32, 68, 55] #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
training_cameras = [1, 2, 3] 

num_joint = 25
max_frame = 300 # Длина отрезка которую мы вычленяем из большого датасета

Есть функция, которая позволяет нам считать данные для каждого класса. 

In [18]:
def read_data(data_path, broken_files_path):
    labels = []
    files = []
    action_classes = {}
    counter = 0
    files_counter = {}

    with open(broken_files_path, 'r') as f:
        broken_files = f.read().split("\n")

    raw_files = os.listdir(data_path)
    num_frames = 0

    for filename in raw_files:
        # if filename.strip('.skeleton') in broken_files:
        # if filename in broken_files:
        #     print('hello')
        # print(filename.strip('.skeleton'))
        if filename.strip('.skeleton') not in broken_files:
            action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
            subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
            camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
            if action_class in training_classes and camera_id in training_cameras:  #and subject_id in training_subjects:
                if action_class in action_classes:
                    if files_counter[action_class] < 120:
                        files.append([filename,action_classes[action_class]])
                        files_counter[action_class] = files_counter[action_class] + 1
                else:
                    action_classes.update({action_class : counter})
                    files_counter.update({action_class : 1})
                    counter+=1
                    files.append([filename,action_classes[action_class]])
#                     labels.append([action_class])
    print("action classes: ", action_classes)
    print("action files: ", files_counter)
    
    return files, action_classes

In [19]:
# Функция фильтр для того, что бы найти только координаты x,y,z(т.к. в датасете
# хранится на порядок больше информации, нежели только координаты x,y,z)
# (Остальные данные нам и не нужны, т.к. у нас нет ик-камер)
def read_skeleton_filter(file):
    with open(file, 'r') as f:
        skeleton_sequence = {}
        skeleton_sequence['numFrame'] = int(f.readline())
        skeleton_sequence['frameInfo'] = []
        for t in range(skeleton_sequence['numFrame']):
            frame_info = {}
            frame_info['numBody'] = int(f.readline())
            frame_info['bodyInfo'] = []

            for m in range(frame_info['numBody']):
                body_info = {}
                body_info_key = [
                    'bodyID', 'clipedEdges', 'handLeftConfidence',
                    'handLeftState', 'handRightConfidence', 'handRightState',
                    'isResticted', 'leanX', 'leanY', 'trackingState'
                ]
                body_info = {
                    k: float(v)
                    for k, v in zip(body_info_key, f.readline().split())
                }
                body_info['numJoint'] = int(f.readline())
                body_info['jointInfo'] = []
                for v in range(body_info['numJoint']):
                    joint_info_key = [
                        'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                        'orientationW', 'orientationX', 'orientationY',
                        'orientationZ', 'trackingState'
                    ]
                    joint_info = {
                        k: float(v)
                        for k, v in zip(joint_info_key, f.readline().split())
                    }
                    body_info['jointInfo'].append(joint_info)
                frame_info['bodyInfo'].append(body_info)
            skeleton_sequence['frameInfo'].append(frame_info)

    return skeleton_sequence

# Здесь мы используем нашу функцию фильр и оформляем дату в x,y,z-cкоординаты
def read_xyz(file, max_body=1, num_joint=25):
    seq_info = read_skeleton_filter(file)
    data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
    for n, f in enumerate(seq_info['frameInfo']):
        for m, b in enumerate(f['bodyInfo']):
            for j, v in enumerate(b['jointInfo']):
                if m < max_body and j < num_joint:
                    data[m, n, j, :] = [v['x'], v['y'], v['z']]

                else:
                    pass

    return data

### В этой функции меняем количество фреймов подаваемых на вход модели

В задании нужно пронаблюдать зависимость качества обучения модели от количества кадров, которые мы подаем в модель. Т.е. то, сколько кадров у нас отведено на один блок. В ноутбуке мы обучали модель на 45 кадрах. Вам предлагается посмотреть в меньшую или большую сторону.

По хорошему функцию ниже нужно прописать внутри определения датасета.

In [20]:
def create_coords_blocks(test_file, chonk_len = 45):   
    frame_counter = 0
    new_labels = []
    new_frames = []
    blocks = []
    
    test_frames = read_xyz(data_path + test_file[0])[0]
    label = test_file[1]
    slice_len = chonk_len * int(len(test_frames)/chonk_len)


    for index in range(len(test_frames[:slice_len])):
        frame_counter += 1
        new_frames.append(test_frames[index].flatten())
        if frame_counter == chonk_len:
            frame_counter = 0
            blocks.append(np.array(new_frames))
            new_labels = new_labels + [label]
            new_frames = []
       
            
    return blocks, new_labels

In [21]:
##### список файлов с лейблами на каждый файл 
working_files_with_labels, action_classes = read_data(data_path, broken_files_path)

action classes:  {3: 0, 7: 1, 24: 2, 27: 3, 32: 4, 38: 5, 55: 6}
action files:  {3: 120, 7: 120, 24: 120, 27: 120, 32: 120, 38: 120, 55: 120}


In [22]:
broken_files_path

'NTU_RGBD_samples_with_missing_skeletons.txt'

In [23]:
LABELS = {v: k for k, v in action_classes.items()}

### Здесь выносится сгенерированный список лейблов с номером класса.

In [24]:
print(LABELS)

{0: 3, 1: 7, 2: 24, 3: 27, 4: 32, 5: 38, 6: 55}


In [25]:
len(working_files_with_labels)

840

In [26]:
import pandas as pd
from  tqdm.notebook import tqdm

In [27]:
data = []
labels = []
##########################################################################
numbers = {x: 0 for x in range(len(action_classes))} #####
##################################################################
for file in tqdm(working_files_with_labels):
    frames_blocks, label = create_coords_blocks(file)
    if label != [] and numbers[label[0]] <= 150:
        numbers[label[0]] = numbers[label[0]] + len(label)
        data = data + frames_blocks
        labels = labels + label
data_np = np.asarray(data)
labels_np = np.asarray(labels)

data_sq = data_np.reshape(len(data_np), -1)
test_data = pd.DataFrame(data_sq)
test_labels = pd.DataFrame(labels_np)
test_data['labels'] = test_labels

  0%|          | 0/840 [00:00<?, ?it/s]

### Ваш итоговый файл для загрузки на колаб

In [ ]:
test_data.to_csv("skeletons_classes_1_30.csv", index = False)